In [1]:
import os
import glob
import time
import requests
from Bio import Entrez, SeqIO, AlignIO
from Bio.Blast import NCBIWWW, NCBIXML

Dans cette cellule, on importe toutes les bibliothèques nécessaires pour le projet :

- `os`, `glob` et `time` : bibliothèques standards de Python pour manipuler les fichiers, rechercher des fichiers dans un dossier, et gérer le temps (utile pour attendre une réponse d'une API).
- `requests` : permet d’envoyer des requêtes HTTP, utilisée ici pour interagir avec l’API web de Clustal Omega (EMBL-EBI).
- `Bio` (Biopython) :
  - `Entrez` : pour accéder aux bases de données NCBI (téléchargement de séquences).
  - `SeqIO` : pour lire et écrire des fichiers FASTA.
  - `AlignIO` : pour lire les fichiers d’alignement générés.
  - `NCBIWWW`, `NCBIXML` : pour effectuer et analyser des recherches BLAST en ligne.

Ces modules constituent les outils principaux pour manipuler des séquences biologiques, interagir avec des bases de données, et automatiser les analyses bioinformatiques.

In [8]:
Entrez.email = 'abdoulwahab.soumare@ucad.edu.sn'

species = ['Homo sapiens', 'Pan troglodytes', 'Gallus gallus']
for specie in species:
    search_term = f'{specie}[Organism] AND COX1[Gene]'
    handle = Entrez.esearch(db = 'nucleotide', term = search_term, retmax = 1)
    record = Entrez.read(handle)
    handle.close()

    if record['IdList']:
        sequence_id = record['IdList'][0]
        print(f'Downloading {specie} : {sequence_id}')

        fetch_handle = Entrez.efetch(db = 'nucleotide', id = sequence_id, rettype = 'fasta', retmode = 'text')
        sequence_record = SeqIO.read(handle = fetch_handle, format = 'fasta')
        fetch_handle.close()

        filename = f'{specie.replace(' ', '_')}_COX1.fasta'
        SeqIO.write(sequence_record, f'data/sequences/{filename}', 'fasta')

Dans cette cellule, on utilise l’outil **Entrez** de Biopython pour interroger la base de données **NCBI Nucleotide** et télécharger les séquences du gène **COX1** (Cytochrome c oxydase subunit 1) pour trois espèces :

- *Homo sapiens* (humain)
- *Pan troglodytes* (chimpanzé)
- *Gallus gallus* (poulet)

Voici les étapes détaillées :

1. `Entrez.email` : requis pour utiliser l’API NCBI (bonne pratique, cela permet de vous contacter en cas d’abus).
2. Pour chaque espèce :
   - On construit une requête de recherche : `"[Organism] AND COX1[Gene]"` pour cibler le bon gène.
   - `Entrez.esearch(...)` cherche la première séquence correspondante (grâce à `retmax=1`).
   - Si un identifiant de séquence est trouvé, on le télécharge au format FASTA via `Entrez.efetch(...)`.
   - On sauvegarde ensuite cette séquence dans un fichier `.fasta` dans le dossier `data/sequences/`, en renommant le fichier selon le nom de l’espèce.

> 🗂️ Exemple de fichier généré : `Homo_sapiens_COX1.fasta`

Ce processus permet d’automatiser la récupération de séquences d'intérêt à partir de la base de données publique NCBI.

In [9]:
files = glob.glob('data/sequences/*.fasta')
sequences = []
for file in files:
    records = list(SeqIO.parse(file, 'fasta'))
    sequences.extend(records)
SeqIO.write(sequences = sequences, handle = 'data/sequences/COX1_combined.fasta', format = 'fasta')

3

Cette cellule a pour but de **rassembler toutes les séquences individuelles** téléchargées précédemment dans un **fichier FASTA combiné**.

Étapes détaillées :

1. `glob.glob('data/sequences/*.fasta')` : recherche tous les fichiers `.fasta` dans le dossier `data/sequences/`.
2. Pour chaque fichier :
   - On lit les séquences avec `SeqIO.parse(...)` et on les ajoute à une liste globale `sequences`.
3. `SeqIO.write(...)` : écrit toutes les séquences dans un nouveau fichier unique appelé `COX1_combined.fasta`.

> 🔄 Ce fichier sera utilisé pour les étapes suivantes d’alignement multiple et d’analyse comparative.

Cela permet de préparer facilement un fichier multi-FASTA compatible avec les outils d'alignement.

In [10]:
with open('data/sequences/COX1_combined.fasta', 'r') as file:
    sequences = file.read()

url = 'https://www.ebi.ac.uk/Tools/services/rest/clustalo/run'
parameters = {
    'email': 'abdoulwahab.soumare@ucad.edu.sn',
    'sequence': sequences,
    'stype': 'dna',
    'outfmt': 'fa'
}

response = requests.post(url = url, data = parameters)
if response.status_code != 200:
    print(f'Submission error : {response.text}')
    exit()

job_id = response.text
print(f'Job submitted with ID : {job_id}')

status_url = f'https://www.ebi.ac.uk/Tools/services/rest/clustalo/status/{job_id}'
while True:
    status = requests.get(url = status_url).text
    print(f'Job status : {status}')

    if status == 'FINISHED':
        break
    
    elif status == 'ERROR':
        print(f'Error during processing')
        exit()

    time.sleep(5)

result_url = f'https://www.ebi.ac.uk/Tools/services/rest/clustalo/result/{job_id}/aln-fasta'
result = requests.get(result_url)

with open('data/sequences/COX1_aligned.fasta', 'w') as file:
    file.write(result.text)

print("Alignment completed and saved in COX1_aligned.fasta")

Job submitted with ID : clustalo-R20250406-163407-0725-34051499-p1m
Job status : RUNNING
Job status : FINISHED
Alignment completed and saved in COX1_aligned.fasta


Dans cette cellule, on effectue un **alignement multiple de séquences ADN** à l’aide de **Clustal Omega**, mais sans avoir besoin de l’installer localement.  
L’alignement est réalisé en appelant l’API REST officielle de l’EMBL-EBI.

#### Étapes détaillées :

1. **Lecture des séquences à aligner** depuis le fichier combiné `COX1_combined.fasta`.

2. **Soumission du travail à l’API Clustal Omega** :
   - L’URL de l’API (`/run`) permet d’envoyer les données pour traitement.
   - Les paramètres incluent :
     - l'adresse e-mail (obligatoire pour l’API),
     - le type de séquences (`stype='dna'`),
     - le format de sortie souhaité (`outfmt='fa'` pour FASTA).

3. **Vérification du statut du job** :
   - On récupère un `job_id` qui identifie le travail soumis.
   - Ensuite, une boucle interroge régulièrement l’API (`/status/{job_id}`) jusqu’à ce que le traitement soit terminé (`status == 'FINISHED'`).

4. **Récupération et sauvegarde du résultat** :
   - Une fois terminé, l’alignement est récupéré via l’URL `/result/{job_id}/aln-fasta`.
   - Le résultat est enregistré dans `COX1_aligned.fasta`.

> 🔁 Ce système asynchrone permet de gérer des alignements en ligne, sans dépendance locale, et de récupérer le résultat automatiquement dès qu’il est prêt.

In [11]:
alignment = AlignIO.read(handle = 'data/sequences/COX1_aligned.fasta', format = 'fasta')
print(f'Length alignment : {alignment.get_alignment_length()}')
for record in alignment:
    print(f'{record.id} : {record.seq[:50]}...')

Length alignment : 751
PQ902116.1 : --------------------------------------------------...
PQ284536.1 : TTATTCGAGCCGAGCTGGGCCAGCCAGGCAACCTTCTAGGTAACGACCAC...
PV053504.1 : --------------------------------------------------...


Dans cette cellule, on charge le fichier FASTA contenant les séquences alignées (`COX1_aligned.fasta`) et on en affiche un aperçu.

#### Étapes détaillées :

1. `AlignIO.read(...)` :
   - Cette fonction de Biopython lit un alignement multiple à partir d’un fichier FASTA.
   - Le résultat est un objet `MultipleSeqAlignment` contenant toutes les séquences alignées.

2. `alignment.get_alignment_length()` :
   - Renvoie la **longueur totale de l’alignement**, c’est-à-dire le nombre de positions (colonnes) dans l’alignement final, y compris les éventuels tirets (`-`) introduits pour l’alignement.

3. La boucle `for record in alignment` :
   - Parcourt chaque séquence alignée.
   - Affiche les **50 premières bases** de chaque séquence pour un aperçu rapide de l’alignement et des régions conservées.

> 🧬 Cette étape est utile pour vérifier que l’alignement a bien été effectué et pour observer visuellement les similarités ou différences entre les séquences.

Dans cette cellule, on effectue une **analyse BLAST (Basic Local Alignment Search Tool)** sur la séquence COX1 de *Homo sapiens* pour identifier des séquences similaires dans la base de données NCBI.

#### Étapes détaillées :

1. `SeqIO.read(...)` :
   - On lit la séquence FASTA de *Homo sapiens* à partir du fichier téléchargé.

2. `NCBIWWW.qblast(...)` :
   - Envoie une requête BLAST en ligne via l’API du NCBI.
   - `program='blastn'` : utilise BLAST pour l'ADN (nucleotide-nucleotide).
   - `database='nt'` : recherche dans la base de données de nucléotides (`nt`).

3. Le résultat est lu et **enregistré dans un fichier XML** (`blast_result.xml`) pour un traitement et une réutilisation ultérieure.

4. `NCBIXML.read(...)` :
   - Analyse le fichier XML de résultats BLAST et renvoie un objet contenant les alignements.

5. La boucle `for alignment in blast_record.alignments[:3]` :
   - Affiche les 3 premiers résultats de l’alignement.
   - Pour chaque résultat, elle affiche :
     - Le **titre de l’alignement** (organisme, gène, etc.),
     - Le **score d’alignement** et le **nombre d’identités** (nucléotides identiques),
     - Les **75 premiers nucléotides** de la requête (`query`), du sujet (`sbjct`), et la ligne d’appariement (`match`), qui indique les similarités visuelles.

> 📌 Cette analyse permet d’identifier des séquences très similaires à la séquence COX1 humaine dans d’autres organismes, utile pour l’étude de la conservation génétique, de la phylogénie, ou pour vérifier l’annotation.

In [ ]:
sequence_record = SeqIO.read(handle = 'data/sequences/Homo_sapiens_COX1.fasta', format = 'fasta')

result_handle = NCBIWWW.qblast(program = 'blastn', database = 'nt', sequence = sequence_record.seq)

with open('data/XML/blast_result.xml', 'w') as out_handle:
    out_handle.write(result_handle.read())

result_handle.close()
with open('data/XML/blast_result.xml') as result_handle:
    blast_record = NCBIXML.read(result_handle)

for alignment in blast_record.alignments[:3]:
    print(f'> {alignment.title}')
    for hsp in alignment.hsps:
        print(f"Score: {hsp.score}, Identity : {hsp.identities}/{hsp.align_length}")
        print(hsp.query[0:75])
        print(hsp.match[0:75])
        print(hsp.sbjct[0:75])
        break

> gi|1530723588|gb|MH120725.1| Homo sapiens isolate K1-56 mitochondrion, complete genome
Score: 1388.0, Identity : 694/694
TTATTCGAGCCGAGCTGGGCCAGCCAGGCAACCTTCTAGGTAACGACCACATCTACAACGTTATCGTCACAGCCC
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
TTATTCGAGCCGAGCTGGGCCAGCCAGGCAACCTTCTAGGTAACGACCACATCTACAACGTTATCGTCACAGCCC
> gi|2419827820|gb|OP682084.1| Homo sapiens isolate CAN0324 mitochondrion, complete genome
Score: 1388.0, Identity : 694/694
TTATTCGAGCCGAGCTGGGCCAGCCAGGCAACCTTCTAGGTAACGACCACATCTACAACGTTATCGTCACAGCCC
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
TTATTCGAGCCGAGCTGGGCCAGCCAGGCAACCTTCTAGGTAACGACCACATCTACAACGTTATCGTCACAGCCC
> gi|2273659072|gb|MZ387509.1| Homo sapiens haplogroup H3 mitochondrion, complete genome
Score: 1388.0, Identity : 694/694
TTATTCGAGCCGAGCTGGGCCAGCCAGGCAACCTTCTAGGTAACGACCACATCTACAACGTTATCGTCACAGCCC
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
TTATTCGAGCCGAGCTGGGCC